In [1]:
push!(LOAD_PATH, "../../src")
using Meteor.TensorNetwork
using Meteor.QuantumCircuit

┌ Info: Precompiling Meteor [top-level]
└ @ Base loading.jl:1260


In [2]:
function random_circuit_1d(L::Int, depth::Int)
    circuit = QCircuit()
    for i in 1:L
        push!(circuit, HGate(i))
    end
    for d in 1:depth
        if d % 2 == 0
            for i in 1:2:L-1
                push!(circuit, CZGate((i, i+1)))
            end
        else
            for i in 2:2:L-1
                push!(circuit, CZGate((i, i+1)))
            end
        end
        for i in 1:L
            push!(circuit, rand([SqrtXGate(i), SqrtYGate(i), TGate(i)]))
        end
    end
    for i in 1:L
        push!(circuit, HGate(i))
    end   
    return circuit
end

L = 20
depth = 10

circuit = random_circuit_1d(L, depth)


QCircuit(AbstractGate[HGate(1), HGate(2), HGate(3), HGate(4), HGate(5), HGate(6), HGate(7), HGate(8), HGate(9), HGate(10)  …  HGate(11), HGate(12), HGate(13), HGate(14), HGate(15), HGate(16), HGate(17), HGate(18), HGate(19), HGate(20)])

In [3]:
# gate fusion
println("total number of gates $(length(circuit)).")
pcircuit = fuse_gate(circuit)
println("number of gates after gate fusion $(length(pcircuit)).")

total number of gates 335.
number of gates after gate fusion 95.


In [4]:
# generate some random basis
N = 5
println("generate $N random basis...")
basis = [rand(0:1, L) for i in 1:5]

generate 5 random basis...


5-element Array{Array{Int64,1},1}:
 [1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0]
 [0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0]
 [1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1]
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0]

In [5]:
#  state vector simulation
state = statevector(ComplexF32, L)
apply!(pcircuit, state)

amps = amplitudes(state, basis, scaling=sqrt(2))

5-element Array{Complex{Float64},1}:
  0.23368513584136993 - 0.29294875264167825im
 -0.05345398187637336 - 0.5593312978744515im
   0.8881338834762585 + 0.20368036627769498im
 0.027851372957229652 + 0.023734837770462067im
   0.1301990598440172 + 0.4933649599552161im

In [6]:
# matrix product states simulation
state = qstate(ComplexF32, L)
apply!(pcircuit, state)

amps = amplitudes(state, basis, scaling=sqrt(2))

5-element Array{Complex{Float32},1}:
   0.23368499f0 - 0.29294878f0im
 -0.053453244f0 - 0.55933124f0im
    0.8881341f0 + 0.20368046f0im
   0.02785147f0 + 0.023734808f0im
   0.13019904f0 + 0.4933649f0im

In [7]:
# PEPS simulation
connectivities = [(i, i+1) for i in 1:L-1]
state = GraphState(ComplexF32, connectivities)
apply!(pcircuit, state)

amps = amplitudes(state, basis, scaling=sqrt(2))

5-element Array{Complex{Float32},1}:
   0.23368528f0 - 0.29294714f0im
 -0.053455766f0 - 0.5593334f0im
    0.8881351f0 + 0.20368269f0im
  0.027847165f0 + 0.02373621f0im
   0.13020033f0 + 0.49336505f0im

In [8]:
amps = amplitudes(state, basis, scaling=sqrt(2), cut=[(3,4)])

5-element Array{Complex{Float32},1}:
   0.23368531f0 - 0.29294714f0im
 -0.053456232f0 - 0.55933356f0im
   0.88813514f0 + 0.20368266f0im
  0.027847365f0 + 0.02373673f0im
   0.13020019f0 + 0.49336475f0im